In [1]:
from darkflow.net.build import TFNet
import cv2
from sort.sort import *

In [10]:
options = {"model": "../yolo_data/cfg/yolo.cfg",
           "load": "../yolo_data/bin/yolo.weights",
           'config': '../yolo_data/cfg',
           'threshold': 0.5, 'summary': None, 'gpu':0.8}

In [11]:
tf_detector = TFNet(options)

Parsing ../yolo_data/cfg\yolo.cfg
Parsing ../yolo_data/cfg/yolo.cfg
Loading ../yolo_data/bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.031080961227416992s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                

In [13]:
import numpy as np
import os

In [31]:
def video_proc(video_file_path, saving_path, tf_detector, max_imgs_per_person=50, delay=1):
    for file in os.listdir(video_file_path):
        print('processing file: {}'.format(file))

        tracker = Sort()

        camera = cv2.VideoCapture(os.path.join(video_file_path,file))
        while(1):
            ret, img_bgr = camera.read()

            if not ret:
                camera.release()
                break

            img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            pred_dict = tf_detector.return_predict(img)
            detections = get_boxes_from_dict(pred_dict)
            bboxes = tracker.update(detections)

            for box_ in bboxes:
                box = box_.astype(int)
                person_id = box[4]
                person_dir = os.path.join(saving_path, str(person_id))
                
                if not os.path.exists(person_dir):
                    os.mkdir(person_dir)
                
                n_img = len(os.listdir(person_dir))
                im_name = str(person_id) + '_' + str(n_img) + '.jpg'
                imCrop = img_bgr[box[1]:box[3], box[0]:box[2]]
                
                if n_img < max_imgs_per_person:
                    cv2.imwrite(os.path.join(person_dir,im_name), imCrop)
                
                cv2.rectangle(img_bgr, (box[0], box[1]), (box[2], box[3]), (255,0,0), 1)
                cv2.putText(img_bgr, 'id:' +str(box[4]),(box[0],box[1]),cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0))
                
            cv2.imshow('ex', img_bgr)
            k = cv2.waitKey(delay)
            if k & 0xFF == ord('q'):
                camera.release()
                break

        if k & 0xFF == ord('q'):
            break
cv2.destroyAllWindows()
    

In [32]:
def get_boxes_from_dict(pred_dict):
    detections = None
    for pred in pred_dict:
        if pred['label'] == 'person':
            bbox = np.array(
                [int(pred['topleft']['x']),
                 pred['topleft']['y'],
                 pred['bottomright']['x'],
                 pred['bottomright']['y'],
                 pred['confidence']])
            if detections is None:
                detections = bbox[np.newaxis,:]
            else:
                detections = np.vstack([detections, bbox])
    if detections is None:
        detections = np.array([])
    return detections

In [33]:
saving_path = 'D:\\data\\pics\\persons_yolo_test'
video_file_path = 'D:\\data\\short\\cam_1'

In [34]:
video_proc(video_file_path, saving_path, tf_detector, max_imgs_per_person=50, delay=1)

processing file: 170814_170101 (1).avi
processing file: 170814_170201 (1).avi


In [35]:
cv2.destroyAllWindows()